# Bring your own components (BYOC)

Starting in V4 Clara train is based of MONAI 
from their website 
"The MONAI framework is the open-source foundation being created by Project MONAI. 
MONAI is a freely available, community-supported, 
PyTorch-based framework for deep learning in healthcare imaging. 
It provides domain-optimized foundational capabilities for developing healthcare imaging training workflows in a native PyTorch paradigm."
<br><img src="screenShots/MONAI.png" alt="Drawing" style="height: 200px;width: 400px"/><br>


Clara Train SDK is modular and flexible enough to allow researchers to bring their own components including:
1. [Transformations](https://docs.monai.io/en/latest/transforms.html#) 
2. [Loss functions](https://docs.monai.io/en/latest/losses.html)
3. [Model Architecture](https://docs.monai.io/en/latest/networks.html)
4. [Loaders](https://docs.nvidia.com/clara/tlt-mi/clara-train-sdk-v3.0/nvmidl/byom.html#bring-your-own-data-loader)
5. [Metrics](https://docs.monai.io/en/latest/metrics.html)

By the end of this notebook you should be able to bring your own components mentioned above.


## Prerequisites
- Familiar with Clara train main concepts. See [Getting Started Notebook](../GettingStarted/GettingStarted.ipynb)
- Nvidia GPU with 8Gb of memory   


## Resources
You could watch the free GTC 2021 talks covering Clara Train SDK
- [Clara Train 4.0 - 101 Getting Started [SE2688]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20101%20Getting%20Started%20%5BSE2688%5D/1_0qgfrql2)
- [Clara Train 4.0 - 201 Federated Learning [SE3208]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20201%20Federated%20Learning%20%5BSE3208%5D/1_m48t6b3y)
- [What’s New in Clara Train 4.0 [D3114]](https://gtc21.event.nvidia.com/media/What%E2%80%99s%20New%20in%20Clara%20Train%204.0%20%5BD3114%5D/1_umvjidt2)
- [Take Medical AI from Concept to Production using Clara Imaging [S32482]](https://gtc21.event.nvidia.com/media/Take%20Medical%20AI%20from%20Concept%20to%20Production%20using%20Clara%20Imaging%20%20%5BS32482%5D/1_6bvnvyg7)
- [Federated Learning for Medical AI [S32530]](https://gtc21.event.nvidia.com/media/Federated%20Learning%20for%20Medical%20AI%20%5BS32530%5D/1_z26u15uk)
- [Get Started Now on Medical Imaging AI with Clara Train on Google Cloud Platform [S32518]](https://gtc21.event.nvidia.com/media/Get%20Started%20Now%20on%20Medical%20Imaging%20AI%20with%20Clara%20Train%20on%20Google%20Cloud%20Platform%20%5BS32518%5D/1_2yjdekmi)
- [Automate 3D Medical Imaging Segmentation with AutoML and Neural Architecture Search [S32083]](https://gtc21.event.nvidia.com/media/Automate%203D%20Medical%20Imaging%20Segmentation%20with%20AutoML%20and%20Neural%20Architecture%20Search%20%5BS32083%5D/1_r5swh2jn)
- [A Platform for Rapid Development and Clinical Translation of ML Models for Applications in Radiology at UCSF [S31619]](https://gtc21.event.nvidia.com/media/A%20Platform%20for%20Rapid%20Development%20and%20Clinical%20Translation%20of%20ML%20Models%20for%20Applications%20in%20Radiology%20at%20UCSF%20%5BS31619%5D/1_oz8qop5a)


## Dataset 
This notebook uses a sample dataset (ie. a single image of a spleen dataset) provided in the package to train a small neural network for a few epochs. 
This single file is duplicated 32 times for the training set and 9 times for the validation set to mimic the full spleen dataset. 
    

# Lets get started
It is helpful to check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi

## 1.1 General Concept
You can easily BYOC into Clara Train SDK by writing your python code then point to it in the `config.json` using `path` instead of the `name` tag. 
Throughout this notebook we have placed all of our examples from our documentations into the [BYOC](BYOC) folder. 

Normal | BYOC  
 --- | ---  
{<br>"name": "CropFixedSizeRandomCenter", <br> "args": {"fields": "image"}<br> } | { <br> "path": "myTransformation.MyAddRandomConstant", <br> "args": {"fields": "image"}<br> } 

 
We modified the [set_env.sh](commands/set_env.sh) to include the path. 
Let us run the cells below that define some helper functions we will be using and see where we added the BYOC to the pythonpath


In [ ]:
MMAR_ROOT="/claraDevDay/GettingStarted/"
print ("setting MMAR_ROOT=",MMAR_ROOT)
%ls $MMAR_ROOT

!chmod 777 $MMAR_ROOT/commands/*
def printFile(filePath,lnSt,lnEnd):
    print ("showing ",str(lnEnd-lnSt)," lines from file ",filePath, "starting at line",str(lnSt))
    !< $filePath head -n "$lnEnd" | tail -n +"$lnSt"

## 1.2 Add BYOC folder to PYTHONPATH 
It is important to add the folder containing your code to the PYTHONPATH variable.
The easiest way to do this is to add it to the `set_env.sh` file since it is called from all the train commands.
Let's take a look at this `set_env.sh` file 

In [ ]:
printFile(MMAR_ROOT+"/commands/set_env.sh",0,20)

## 2.1 BYO Transformation: Adding random value to image pixels
Now lets write a full transformation `MyAddRandomConstant` from scratch. For this you need to:
1. Implement `MultiFieldTransformer`
2. Define `transform` and `is_deterministic` functions

see how we did this in by running the cell below

In [ ]:
printFile(MMAR_ROOT+"/BYOC/myTransformation.py",16,30)


Now to run this we need to modify the train config by setting the `path` 
to our newly created transformation `myTransformation.MyAddRandomConstant`. 
We also would like to debug the output so we will add the `SaveAsNifti` Transform. 
This transform would pause the training and save batches to `out_dir` for us to check. 
Note: we set `"interrupt": false` in the example however, 
when debugging you may want to change it to true so that you would get a pause to check the batches then continue to generate more 

In [ ]:
configFile=MMAR_ROOT+"/config/trn_BYOC_transform.json"
printFile(configFile,89,10)


## 2.2 Run and see Debugging Data
So let us now run training and see the results 

In [ ]:
! $MMAR_ROOT/commands/train_W_Config.sh trn_BYOC_transform.json


Now let us see the sample images in the debug folder  

In [ ]:
! ls -la /claraDevDay/Data/_tmpDebugPatches/


## 3. BYO Network Architecture and Loss
Clara Train SDK also allows you to write your own network architecture as well as your loss function. 
In this example we have a shallow Unet architecture defined in [myNetworkArch.py](BYOC/myNetworkArch.py) 
as well as our own dice loss defined in [myLoss.py](BYOC/myLoss.py). 

Normal | BYOC  
 --- | --- 
"loss": {<br> "name": "DiceLoss",<br> "args":{ ...      } <br>}, | "loss": {<br> **"path"**: "myLoss.MyDiceLoss",<br>  "args": {... }<br>} |
"model": {<br> "name": "UNet",<br>"args": { ... }<br>}, | "model": {<br>**"path"**: "myNetworkArch.MyBasicUNet",<br>"args": { ... }<br>},
 

Let us see how it is defined   

In [ ]:
printFile(MMAR_ROOT+"/BYOC/myNetworkArch.py",0,30)
printFile(MMAR_ROOT+"/BYOC/myLoss.py",0,30)

Let us Examine the config file [trn_BYOC_Arch_loss.json](config/trn_BYOC_Arch_loss.json) 

In [ ]:
configFile=MMAR_ROOT+"/config/trn_BYOC_Arch_loss.json"
printFile(configFile,11,18)
printFile(configFile,32,43)


Now let us train our network  

In [ ]:

! $MMAR_ROOT/commands/train_W_Config.sh trn_BYOC_Arch_loss.json


## 4. BYO Data Loader
For this example we will see how to use a custom loader specifically to load a numpy file. 
To do this, we first load our nii.gz file and save it a np.  

In [ ]:
import nibabel as nib
import numpy as np
dataRoot="/claraDevDay/Data/sampleData/"
for imgLb in ["imagesTr","labelsTr"]:
    filename= dataRoot+imgLb+"/spleen_8.nii.gz"
    img = nib.load(filename)
    data = img.get_fdata()
    np.save(filename[:-7]+".npy",data)
!ls -ls $dataRoot/imagesTr
!ls -ls $dataRoot/labelsTr

Now that we have a numpy files, we will change the [trn_BYOC_DataLoader.json](config/trn_BYOC_DataLoader.json) 
to point to MyNumpyLoader. Let us take a look at the configuration file changes

In [ ]:
configFile=MMAR_ROOT+"/config/trn_BYOC_DataLoader.json"
printFile(configFile,23,9)

Now let us check the code written for loading from numpy in the [myNpLoader.py](BYOC/myNpLoader.py) file.

In [ ]:
configFile=MMAR_ROOT+"/BYOC/myNpLoader.py"
printFile(configFile,0,27)



Now let us run training by running the cell below

In [ ]:
! $MMAR_ROOT/commands/train_W_Config.sh trn_BYOC_DataLoader.json "/claraDevDay/sampleData/dataset_28GB_npy.json"


# Exercise

1. Modify the custom loss file to be a weighted dice loss per label.
Tips:
    1. Code below is already in the [myLoss.py](BYOC/myLoss.py) after the is skip background if statement, you just need to uncomment it
    ```
    if label_weights is not None:
        label_weights = label_weights[1:]
    ```
    2. Similarly uncomment the lines below that are already in the same function to multiply the weights given with the loss  
    ``` 
    if label_weights is not None:  add wights to labels
        print("========== Debug research intersection.shape=", intersection.shape)
        w = tf.constant(label_weights, dtype=tf.float32)
        intersection = tf.multiply(w, intersection)
        print("========== Debug research intersection.shape=", intersection.shape)
    
    ```
    3. You need to pass the weights into these functions
2. You can use code in [myMetric.py](BYOC/myMetric.py) and customize it to bring your own metric
